# Modelling NIQS Hardware TF

In [1]:
import sys
sys.path.insert(0, '../../src/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import torch.optim as optim
import tensorflow as tf

from qiskit.quantum_info import DensityMatrix
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from cost_functions import *
from optimization import *
from quantum_maps import *
from quantum_tools import *
from src_torch import *


#from src_tf import *

In [2]:
def apply_map(state, kraus_list):
    state = [K@state@tf.linalg.adjoint(K) for K in kraus_list]
    state = tf.math.reduce_sum(tf.stack(state), axis=0)
    return state


def expectation_value(state, observable):
    ev = tf.linalg.trace(observable@state)
    return ev

In [15]:
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n
rank = 16

A = tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64)
B = tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64)
G = A + 1j*B
Q, R = tf.linalg.qr(G, full_matrices = False)
D = tf.linalg.tensor_diag_part(R)
D = tf.math.sign(D)
D = tf.linalg.diag(D)
U = Q@D


kraus_target_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]

In [16]:
N = 100
state_index, observ_index = index_generator(n, N, trace=False)

input_list = []
circuit_list = []
for i, j in zip(state_index, observ_index):

    config = numberToBase(i, 6, n)
    state = tf.convert_to_tensor(prepare_input(config), dtype = tf.complex64)

    config = numberToBase(j, 3, n)
    observable = tf.convert_to_tensor(pauli_observable(config),  dtype = tf.complex64)
    
    input_list.append([state, observable])

target_list = [expectation_value(apply_map(input[0], kraus_target_list), input[1]) for input in input_list]
optimizer = tf.optimizers.Adam(learning_rate=0.05)

A = tf.Variable(tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64), trainable=True)
B = tf.Variable(tf.cast(tf.random.normal((rank*d, d), 0, 1), dtype=tf.complex64), trainable=True)

In [17]:
for i in range(200):
    with tf.GradientTape() as tape:
        G = A + 1j*B
        Q, R = tf.linalg.qr(G, full_matrices = False)
        D = tf.linalg.tensor_diag_part(R)
        D = tf.math.sign(D)
        D = tf.linalg.diag(D)
        U = Q@D

        kraus_model_list =  [U[i*d:(i+1)*d, :d] for i in range(rank)]
        pred_list = [expectation_value(apply_map(input[0], kraus_model_list), input[1]) for input in input_list]
        loss = tf.math.reduce_mean(tf.stack([(target - predicted)**2 for target, predicted in zip(target_list, pred_list)]))

    grads = tape.gradient(loss, [A,B])
    optimizer.apply_gradients(zip(grads, [A, B]))
    
    print(f"loss: {loss: .4f}, {tf.math.reduce_mean(tf.math.abs(G))}")

loss:  0.0136-0.0000j, 1.2794349193572998
loss:  0.0088-0.0000j, 1.2832427024841309
loss:  0.0053-0.0000j, 1.2886931896209717
loss:  0.0030-0.0000j, 1.2944152355194092
loss:  0.0016-0.0000j, 1.301639437675476
loss:  0.0009+0.0000j, 1.3092080354690552
loss:  0.0007+0.0000j, 1.3172378540039062
loss:  0.0007+0.0000j, 1.325500726699829
loss:  0.0009+0.0000j, 1.3342914581298828
loss:  0.0010+0.0000j, 1.3429569005966187
loss:  0.0011+0.0000j, 1.351057529449463
loss:  0.0012+0.0000j, 1.3587851524353027
loss:  0.0012+0.0000j, 1.3664116859436035
loss:  0.0011-0.0000j, 1.3737260103225708
loss:  0.0010-0.0000j, 1.38079833984375
loss:  0.0009+0.0000j, 1.3874156475067139
loss:  0.0008-0.0000j, 1.3936686515808105
loss:  0.0007-0.0000j, 1.3993558883666992
loss:  0.0005+0.0000j, 1.4045956134796143
loss:  0.0004+0.0000j, 1.4096460342407227
loss:  0.0003+0.0000j, 1.4145019054412842
loss:  0.0003-0.0000j, 1.4191725254058838
loss:  0.0002-0.0000j, 1.4235974550247192
loss:  0.0002+0.0000j, 1.42770791053771

loss:  0.0000-0.0000j, 1.4837985038757324
loss:  0.0000-0.0000j, 1.4837987422943115
loss:  0.0000+0.0000j, 1.4837989807128906
loss:  0.0000-0.0000j, 1.4837994575500488


## Test

In [ ]:
n = 2
d = 2**n

state_input_list = [prepare_input(numberToBase(i, 6, n)) for i in range(6**n)]

np.random.seed(43)
tf.random.set_seed(43)

optimizer = tf.optimizers.SGD(learning_rate=1e-8)

X_target = generate_ginibre(d**2, 2)
state_target_list = [apply_map(state_input, choi_target) for state_input in state_input_list]

X_model = generate_ginibre(d**2, 2)

for i in range(100):
    state_input = state_input_list[0]
    state_target = state_target_list[0]
    print("param", grads[0,0])
    
    with tf.GradientTape() as tape:
        tape.watch(X_model)
        choi_model = generate_choi(X_model)
        state_model = apply_map(state_input, choi_model)
        fid = -state_fidelity(state_model, state_target)
    
    grads = tape.gradient(fid, X_model)
    print(grads)
    #optimizer.apply_gradients(zip([grads], [X_model]))
    #print("fid", fid)
    